In [1]:
import requests
import pandas as pd
import numpy as np
import re

In [2]:
# создание списка компаний
# сбор данных по каждой компании
# анализ собранных данных


# Создание списка компаний

In [26]:
url_list = 'https://www.testfirm.ru/rating' # рейтинг по всем предприятиям / #'https://www.testfirm.ru/rating/63/sverdlovskaya-oblast/'
k_1 = 63   #код деятельности
k_2 = "sverdlovskaya-oblast"  #область
url_list_copm = url_list + '/' + str(k_1) + '/' + k_2 + '/'
text = requests.get(url_list_copm) # 
try:
  text.status_code == 200
  print('Список загружен нормально')
except:
  print('ОШИБКА! Проверяй!')

Список загружен нормально


In [47]:
# поиск названия компании
text_write = text.text

def find_komp(text_write, N):
    """ф. для поиска ИНН и названия по каждой компании - для формирования url-ссылки на информацию по этой компании
    """
    firma_list = text_write.split('org-link" href="/result/')
    return firma_list[N].split('"')[0]

List_all_komp = [] # список для url для каждой компании
for i in range(1, 51): # страница содержит список из 50 компаний на странице
    b = find_komp(text_write, i) 
    List_all_komp.append(b) # 
    print(i,'\t', List_all_komp[-1])
def find_komp_re(text_write, N):
  """  ф. для поиска ИНН и названия по каждой компании - с помощью регулярных выражений
  """
  pass    

1 	 6671042458_ooo-bit
2 	 6671332911_ao-regionalnyy-informatsionnyy-tsentr
3 	 6670320529_ooo-miralodzhik-it
4 	 6673240328_ooo-sertum-pro
5 	 6672258403_ooo-tekhkonsur-ooo-tekhkonsur
6 	 6673204560_ooo-ekspertbizneskonsalting
7 	 6670020959_ooo-kae-ekspert
8 	 6674301774_ooo-kompaniya-informatsionnykh-tekhnologiy
9 	 6660075589_ooo-siams
10 	 6678074441_ooo-ekstensiv-servis
11 	 6670419574_ooo-smartpeydzh
12 	 6659208481_ooo-boksberri-region
13 	 6671218373_ooo-konsultantplyus-ekaterinburg
14 	 6670237020_ooo-ntssoft
15 	 6671291239_ao-informatsionnaya-set
16 	 6671390568_ooo-plm-inzhiniring
17 	 6671341000_ooo-praymer
18 	 6658372383_ooo-konsultantplyusservis
19 	 6671142678_ooo-internet-pro
20 	 6625030240_pervouralskoe-mup-edinyy-raschetnyy-tsentr
21 	 6679077822_ooo-keysistems-ural
22 	 6670418517_ooo-profbizneskonsalting
23 	 6673174629_ooo-konsultant-sverdlovskaya-oblast
24 	 6674238593_ooo-konsultantplyus-sverdlovskaya-oblast
25 	 6672185480_ooo-ayteks
26 	 6658280140_ooo-stra

# Сбор данных по каждой компании

In [ ]:
# зайдем на страницу по каждой компании и начнем сбор данных
def find_info_comp(List_all_komp, n_comp):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку
    """
    url_comp = 'https://www.testfirm.ru/result/'+ List_all_komp[n_comp]
    text = requests.get(url_comp)
    try:
        text.status_code == 200
#        print('Готов!')
    except:
        print('Ошибка, проверяй')
    stroka = [] # собираем информацию сюда
    text_write = text.text
    a1 = text_write.split('Организация: ')[1]
 #   stroka.append(a1.replace(' &quot;', '').split('<')[0].split(' ')[0]) # правовая форма
    if text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0]: # название
        stroka.append(text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0])
    else:
        stroka.append(0)
#    print(stroka[-1])
    stroka.append(a1.split('ИНН: ')[1][:10]) # ИНН
#    print(stroka[-1])
    stroka.append(a1.split('<b>')[2].split('<')[0].rstrip()) # активы
#    print(stroka[-1])
    stroka.append(a1.split('Активы на ')[1].split('/b>')[1][:5].rstrip()) # млн или тыс
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[1].split('<')[0].rstrip()) # выручка
#    print(stroka[-1])
    stroka.append(a1.split('Выручка')[1].split('b>')[2].split('руб')[0].rstrip()) # млн ии тыс
#    print(stroka[-1])
    if 'численности ' in text_write:
        stroka.append(a1.split('численности ')[1].split('b>')[1].split('<')[0]) # численность человек
#        print(stroka[-1])
    else:
        stroka.append(0) 
    if 'Дополнительные данные ФНС' in text_write:
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[1].split('<')[0].rstrip()) # доходы 
#        print(stroka[-1])
        stroka.append(a1.split('Дополнительные данные ФНС')[1].split('padding-left:12px"><b>')[2].split('<')[0].rstrip()) # расходы
#        print(stroka[-1])
    else:
        stroka.append(0)
        stroka.append(0)
    if "Уплачено налогов" in text_write:
        stroka.append(text_write.split("Уплачено налогов")[1].split('b>')[1].split('<')[0].rstrip()) # налоги уплаченные
    else:
        stroka.append(0)
    return stroka

# пример -
print(find_info_comp(List_all_komp, 19))

def find_info_comp_re(List_all_komp, n_comp):
    """ ф.собирает информацию по каждой компании и сохраняет данные в строку с помощью регулярных выражений
    """
    pass


# Тренировка поиска

In [132]:
# загрузка данных компании
url = 'https://www.testfirm.ru/result/6671291239_ao-informatsionnaya-set'
text = requests.get(url) # 
try:
  text.status_code == 200 # получен успешный ответ от сервера
  print('Жги дальше')
except: # 
  print('ОШИБКА! Проверяй!') 

Жги дальше


In [ ]:
#print(text.status_code) # код ответ - получен успешный ответ от сервера
#print(text.content) # покажет информацию в виде байтов - если надо найти картинку
#print('Кодировка - ', text.encoding)

#text_write = text.text
#print(text_write.split('Организация: ')[1][:1000])

#print(text_write.split('id="searchText" name="s" value=')[1].split('placeholder')[0].rstrip())
#print(text.text) # покажет весь текст - код страницы
#text.headers['Content-Type'] # покажет метаданные(служебные данные) в виде словаря

In [ ]:
# пример поиска
#№url = 'https://www.testfirm.ru/result/6625030240_pervouralskoe-mup-edinyy-raschetnyy-tsentr'
#text = requests.get(url) #
#print(text.text.split('Дополнительные данные ФНС')[1])

# Заполнение таблицы 

In [136]:
Table_find_values = [] # таблица заполняется найденными данными
for i in range(len(List_all_komp)):
    Table_find_values.append(find_info_comp(List_all_komp, i))
    print(i)    # проверка на каком значении произойдет ошибка

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [98]:
#List_all_komp[19]

'6625030240_pervouralskoe-mup-edinyy-raschetnyy-tsentr'

# Анализ данных

In [139]:
data = pd.DataFrame(Table_find_values)
data.columns = ['Наименование', 'ИНН', 'Активы', 'ед.изм.', 'Выручка', 'ед.изм.', 'Численность персонала', 'Доходы', 'Расходы', 'Налоги'] 
data.head()

,Наименование,ИНН,Активы,ед.изм.,Выручка,ед.изм.,Численность персонала,Доходы,Расходы,Налоги
0,"""БИТ""",6671042458,191,млн.,564,млн.,17,566,515,58
1,"""Региональный Информационный Центр""",6671332911,252,млн.,478,млн.,518,530,630,105
2,"""МираЛоджик ИТ""",6670320529,"87,8",млн.,444,млн.,318,444,435,"30,8"
3,"""СЕРТУМ-ПРО""",6673240328,854,млн.,414,млн.,71,443,384,"40,1"
4,"""ТЕХКОНСУР ООО ТЕХКОНСУР""",6672258403,171,млн.,351,млн.,52,352,327,"77,9"
5,"""ЭКСПЕРТБИЗНЕСКОНСАЛТИНГ""",6673204560,"57,2",млн.,259,млн.,32,260,164,"12,4"
6,"""КАЕ ЭКСПЕРТ""",6670020959,192,млн.,223,млн.,11,231,206,"13,5"
7,"""КОМПАНИЯ ИНФОРМАЦИОННЫХ ТЕХНОЛОГИЙ""",6674301774,148,млн.,219,млн.,25,221,"41,2","8,54"
8,"""СИАМС""",6660075589,"96,8",млн.,199,млн.,2,203,197,"15,1"
9,"""ЭКСТЕНСИВ-СЕРВИС""",6678074441,"21,5",млн.,176,млн.,47,357,294,12
